# import Library

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
from collections import OrderedDict #for multiGPU
from tqdm import tqdm

import os
import random
import time

# Load Data

In [ ]:
print("loading dataset")

data_transforms = {
    'train' : transforms.Compose([ #horizontal flip, random crop, normalization           
            #transforms.ToPILImage(),
            # transforms.Resize((256,256)),
            transforms.RandomCrop(32, padding = 4), #augmentation, input for ResNet
            transforms.RandomHorizontalFlip(), #augmentation
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            #transforms.Normalize((0.4914, 0.4822, 0.4465), (1, 1, 1)), #with the per-pixel mean subtracted

        ]) , 
        'test': transforms.Compose([
            transforms.ToTensor(), #single view of the original 32x32 image
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
    }
#access data_transforms['train']

#dataloader (batch)
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform = data_transforms['train']
)
trainloader = torch.utils.data.DataLoader(
    train_set, batch_size = 128, shuffle = True, num_workers=2
)

test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=data_transforms['test'])
testloader = torch.utils.data.DataLoader(
    test_set, batch_size=100, shuffle=False, num_workers=2)

#classes
classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')


loading dataset


100%|██████████| 170498071/170498071 [00:02<00:00, 82378232.55it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Model Architecture

In [ ]:
#classes for Plain20 Model
class conv16 (nn.Module): #Blue
  def __init__(self):
    super().__init__()
    self.Fx1 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16),
        nn.ReLU()
    )
    self.Fx2 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16),
        nn.ReLU()
    )
    self.Fx3 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16),
        nn.ReLU()
    )
    self.relu = nn.ReLU()
    
  def forward(self,x):
    x1 = self.Fx1(x) 
    x2 = self.Fx2(x1) 
    x3 = self.Fx3(x2)
    x = self.relu(x3)
    return x

#classes for Plain20 Model
class conv32 (nn.Module): #Purple
  def __init__(self):
    super().__init__()
    self.Fx1 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(16, 32, kernel_size = 3, stride = 2, padding = 1, bias = False),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )
    # self.residual_mapping = nn.Sequential(
    #     nn.Conv2d(16,32,kernel_size =1, stride = 2, bias= False),
    #     nn.BatchNorm2d(32)
    # )
    self.Fx2 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )
    self.Fx3 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32),
        nn.ReLU()
    )
    self.relu = nn.ReLU()
    
  def forward(self,x):
    # print(x.shape)
    # print(self.Fx1(x).shape)
    # print(self.residual_mapping(x).shape)
    x1 = self.Fx1(x) 
    x2 = self.Fx2(x1) 
    x3 = self.Fx3(x2)
    x = self.relu(x3)
    return x

#classes for Plain20 Model
class conv64 (nn.Module): #Green
  def __init__(self):
    super().__init__()
    self.Fx1 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(32, 64, kernel_size = 3, stride = 2, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    # self.residual_mapping = nn.Sequential(
    #     nn.Conv2d(32,64,kernel_size =1, stride = 2, bias= False),
    #     nn.BatchNorm2d(64)
    # )
  
    self.Fx2 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    self.Fx3 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )
    self.relu = nn.ReLU()
    
  def forward(self,x):
    x1 = self.Fx1(x) 
    x2 = self.Fx2(x1) 
    x3 = self.Fx3(x2) 
    x = self.relu(x3)
    return x


In [ ]:
class MyModel(nn.Module): #Plain20 architecture implementation
    def __init__(self, conv2 = conv16, conv3 = conv32,
            conv4 = conv64, num_classes = 10, init_weights = True): #num_classes
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding=1, bias = False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        self.conv2 = conv2()
        self.conv3 = conv3()
        self.conv4 = conv4()
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fclayer = nn.Linear(64, num_classes)

        if init_weights:
            self.initialize_weights()
    
    # define weight initialization function
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0) 
        
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x = self.avg_pool(x4)
        x = x.view(x.size(0), -1)
        output = self.fclayer(x)
        return output

In [ ]:
#device: cpu -> no error

from torchsummary import summary
resnet20 = MyModel()
summary(resnet20, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,304
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
              ReLU-9           [-1, 16, 32, 32]               0
           Conv2d-10           [-1, 16, 32, 32]           2,304
      BatchNorm2d-11           [-1, 16, 32, 32]              32
             ReLU-12           [-1, 16, 32, 32]               0
           Conv2d-13           [-1, 16, 32, 32]           2,304
      BatchNorm2d-14           [-1, 16,

In [ ]:
# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
#for colab
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is ", device)

device is  cuda


# Train

In [ ]:
# #get_model when loading model parameters trained w/ multi-gpu
def get_model(model_name, checkpoint_path):
    print("start loading model parameter")
    model = model_name()
    state_dict = torch.load(checkpoint_path)

    new_state_dict = OrderedDict()

    for k, v in state_dict.items():
        name = k.replace('module.', '')
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    print("model parameter loaded")
    return model

In [ ]:

def train(n_epochs, train_loader) :
 
    epochs = n_epochs
    train_dataloader = train_loader
    #get current time -> folder name
    train_date = time.strftime("%y%m%d_%H%M", time.localtime(time.time()))
    # print(train_date)

    #log path for tensorboard
    log_path = os.path.join(os.getcwd(),"logs",train_date)
    print("log_path is {}".format(log_path))

    #model path to save '.pth'file
    model_path = os.path.join(os.getcwd(),"models",train_date)
    print("model_path is {}".format(model_path))
    if os.path.isdir(model_path) == False:
            os.makedirs(model_path)

    #writer for tensorboard
    writer = SummaryWriter(log_path)
 
    # whether available GPU
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
    
    #define model (처음부터 학습시)
    print("처음부터 학습")
    model = MyModel()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    ####### 이전 학습한거에 추가적으로 학습시
    # print("이전에 하던거 부터 학습")
    # model_name = MyModel
    # # /content/models/230412_2145/0.pth
    # checkpoint_path = './12.pth'  #수정 필요
    # model = get_model(model_name,checkpoint_path)
    # model.to(device)

    #######

    #define loss funcion, optimizer, (scheduler)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(),lr=0.1, momentum=0.9,weight_decay=0.0001)
    #lr = 0.025, momentum = 0.9, weight_decay = 0.0005(original)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [81,122], gamma = 0.1)
    # 0.1 -> 0.01 (32K iterations = 81 epochs) -> 0.001 (48K iterations = 122 epochs)

    print("start training")
    #Train
    for epoch in range(epochs):
        
        model.train(True)
        len_trainbatch = len(train_dataloader)

        i = 0
        for inputs, targets in tqdm(train_dataloader):
            i = i+1
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            writer.add_scalar('train/loss', loss.item(), epoch*len_trainbatch+i) #iterations

        scheduler.step()    
        print("train loss of {}th epoch is {}".format(epoch,loss.item())) ## 수정 필요
 
        # Save every N(30) epoch
        save_epoch = 30
        if save_epoch > 0 and epoch % save_epoch == save_epoch-1:
            state_dict = model.state_dict()
            torch.save(state_dict, os.path.join(model_path, str(epoch+1)+'.pth')) #epoch+1로 바꾸기
        
    print('Finished Training')

    #save model after training
    torch.save(model.state_dict(), './model.pth') 

   

# Main

In [ ]:

def main() :

    random.seed(53)
    train(163, trainloader) #64k iterations = 163 epochs

   

if __name__ == '__main__':
    main()

log_path is /content/logs/230416_0801
model_path is /content/models/230416_0801
처음부터 학습
start training


100%|██████████| 391/391 [00:31<00:00, 12.48it/s]


train loss of 0th epoch is 1.6721229553222656


100%|██████████| 391/391 [00:24<00:00, 16.12it/s]


train loss of 1th epoch is 0.9643071889877319


...

train loss of 160th epoch is 0.06619148701429367


100%|██████████| 391/391 [00:24<00:00, 16.08it/s]


train loss of 161th epoch is 0.0646500438451767


100%|██████████| 391/391 [00:24<00:00, 15.94it/s]


train loss of 162th epoch is 0.04269452020525932
Finished Training


# Test

In [ ]:
def test():

    ####### not touch
    model_name = MyModel
    # /content/models/230412_2145/0.pth
    #/content/models/230412_2208/1.pth
    #/content/models/230416_0801/150.pth
    checkpoint_path = './models/230416_0801/90.pth'  #수정 필요
    mode = 'test' 
#     data_dir = "./test_data"
#     meta_path = "./answer.json"
    model = get_model(model_name,checkpoint_path)
    #######
    
#     # Create training and validation datasets
#     test_datasets = MyDataset(meta_path, data_dir, data_transforms['test'],is_train= False) #self,meta_path,root_dir,transform=None, is_train = True

#     # Create training and validation dataloaders
#     test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=False, num_workers=4)

    test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=data_transforms['test'])
    
    testloader = torch.utils.data.DataLoader(
    test_set, batch_size=100, shuffle=False, num_workers=2)
    
    # Detect if we have a GPU available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
    
    # Send the model to GPU
    #multi-GPU
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)

    # Set model as evaluation mode
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    
    # Inference
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        batch_idx = 0
        for inputs, targets in tqdm(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_idx = batch_idx + 1
            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))



# Main

In [ ]:

def main() :

    random.seed(53)
    test()

   
if __name__ == '__main__':
    main()

start loading model parameter
model parameter loaded
Files already downloaded and verified


  5%|▌         | 5/100 [00:00<00:04, 20.69it/s]

1 100 Loss: 0.149 | Acc: 92.000% (92/100)
2 100 Loss: 0.225 | Acc: 91.000% (182/200)
3 100 Loss: 0.257 | Acc: 90.000% (270/300)
4 100 Loss: 0.269 | Acc: 90.500% (362/400)
5 100 Loss: 0.277 | Acc: 90.600% (453/500)
6 100 Loss: 0.257 | Acc: 91.500% (549/600)
7 100 Loss: 0.258 | Acc: 91.286% (639/700)


 11%|█         | 11/100 [00:00<00:03, 25.77it/s]

8 100 Loss: 0.279 | Acc: 91.000% (728/800)
9 100 Loss: 0.292 | Acc: 90.778% (817/900)
10 100 Loss: 0.284 | Acc: 90.900% (909/1000)
11 100 Loss: 0.293 | Acc: 90.727% (998/1100)
12 100 Loss: 0.314 | Acc: 90.500% (1086/1200)
13 100 Loss: 0.312 | Acc: 90.462% (1176/1300)


...

84 100 Loss: 0.326 | Acc: 90.345% (7589/8400)
85 100 Loss: 0.326 | Acc: 90.329% (7678/8500)
86 100 Loss: 0.328 | Acc: 90.267% (7763/8600)
87 100 Loss: 0.327 | Acc: 90.276% (7854/8700)
88 100 Loss: 0.327 | Acc: 90.273% (7944/8800)
89 100 Loss: 0.327 | Acc: 90.281% (8035/8900)
90 100 Loss: 0.327 | Acc: 90.300% (8127/9000)
91 100 Loss: 0.326 | Acc: 90.286% (8216/9100)


 98%|█████████▊| 98/100 [00:03<00:00, 31.86it/s]

92 100 Loss: 0.325 | Acc: 90.315% (8309/9200)
93 100 Loss: 0.326 | Acc: 90.258% (8394/9300)
94 100 Loss: 0.326 | Acc: 90.277% (8486/9400)
95 100 Loss: 0.325 | Acc: 90.263% (8575/9500)
96 100 Loss: 0.326 | Acc: 90.240% (8663/9600)
97 100 Loss: 0.324 | Acc: 90.299% (8759/9700)
98 100 Loss: 0.324 | Acc: 90.316% (8851/9800)


100%|██████████| 100/100 [00:03<00:00, 30.16it/s]

99 100 Loss: 0.325 | Acc: 90.293% (8939/9900)
100 100 Loss: 0.325 | Acc: 90.300% (9030/10000)
